<a href="https://colab.research.google.com/github/alecinvan/Colab_AI_tutorial/blob/main/personalknowledgebase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="blue" size="6"> ⭐*Alec 作品*

<font color="darkred" size="6">
本套代码实现的是一个“个人知识库问答 + 学习助手”的完整最小闭环：

*   你把课程资料、笔记、项目文档或简历放进 kb/ 目录；
*   系统先把文档解析成可管理的小段（chunk），再用 embedding 模型把每个段落转成向量，并用 FAISS 建一个向量索引。这样当你提问时，它不会“凭空瞎猜”，而是先到索引里检索出最相关的几段原文作为证据，再把这些证据连同你的问题一起交给大语言模型生成答案，并在回答中标注引用来源，从而形成一种“像有记忆”的对话体验。
*   在此基础上，代码还提供了学习场景的增值能力：一键生成 7 天游学复习计划、自动出测验题和闪卡，把“资料→理解→练习”的链路串起来。
*   最后用 Gradio 把这一整套流程封装成一个可交互的网页应用，包含“上传资料—构建知识库—对话问答—复习计划—测验/闪卡”五个入口。
*   要使用这个Demo：先在轻量环境跑通效果，再把大规模建库/批量出题等重任务提交到算力网执行，任务完成后将索引与产物回传到工作目录继续使用与迭代。

# <font color="blue" >▶️ 安装依赖

In [5]:
import warnings
warnings.filterwarnings('ignore')

!pip -q install numpy==2.0.0 websockets==15.0.1 gradio openai==1.40.0 python-dotenv==1.0.1 \
  sentence-transformers faiss-cpu pypdf==4.3.1 \
  transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 48.2 MB/s eta 0:00:00


<font color="Darkgreen" size="4">解释（依赖用途）：

*   sentence-transformers：把文本变成向量（embedding）
*   faiss-cpu：向量检索库（快速查相似段落）
*   pypdf：解析 PDF
*   transformers/accelerate：本地加载开源小模型用于回答/出题/计划


In [6]:
!nvidia-smi

Wed Dec 31 03:38:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              8W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# <font color="blue" >▶️ 创建一个最小可运行 Demo 数据
（用户可换成自己的资料）

In [7]:
import os, textwrap, pathlib

os.makedirs("kb", exist_ok=True)

(pathlib.Path("kb") / "sample_course_notes.md").write_text(textwrap.dedent("""
# 机器学习复习笔记（节选）
## 1. 过拟合与正则化
过拟合：模型在训练集上表现很好，但在测试集上变差。
常见应对：L2 正则、早停、数据增广、交叉验证。

## 2. 偏差-方差
偏差高：模型太简单（欠拟合）。
方差高：模型太复杂（过拟合）。
目标：在偏差与方差之间权衡。

## 3. SVM 核技巧
核函数将数据映射到高维空间，使线性可分。
常用核：RBF，高斯核。
""").strip(), encoding="utf-8")

(pathlib.Path("kb") / "sample_project.md").write_text(textwrap.dedent("""
# 项目：校园课程资料问答助手
目标：把课程笔记与项目文档变成可对话知识库。
核心流程：资料解析 -> 切分 -> 向量化建库 -> 检索增强问答（RAG）-> 生成复习计划/测验/闪卡。
""").strip(), encoding="utf-8")

(pathlib.Path("kb") / "sample_resume.md").write_text(textwrap.dedent("""
# 简历（节选）
- 擅长：Python、数据分析、机器学习基础
- 项目经验：做过一个“课程知识库问答助手”，支持资料上传、对话问答、自动出题、复习计划生成。
""").strip(), encoding="utf-8")

print("已创建样例资料：", sorted([p.name for p in pathlib.Path("kb").glob("*")]))


已创建样例资料： ['sample_course_notes.md', 'sample_project.md', 'sample_resume.md']
